<a href="https://colab.research.google.com/github/PranavM98/BERT---Multiclass-Sentiment-Analysis/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch

In [2]:
import torch 
import pandas as pd
from tqdm.notebook import tqdm

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df=pd.read_csv('/content/drive/My Drive/NLP_Final_Project/smile-annotations-final.csv',
              names=['id','text','category'])

In [10]:
df.set_index('id',inplace=True)

In [11]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [12]:
df=df[df['category']!='nocode']

In [13]:
df=df[~df['category'].str.contains('\|')]

In [14]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [15]:
possible_labels=df.category.unique()

In [16]:
label_dict={}
for index,possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [17]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [18]:
df['label']=df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [19]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(
                df.index.values,
                df.label.values,
                test_size=0.15,
                stratify=df.label.values
)

In [20]:
df['data_type']=['not_set']*df.shape[0]

In [21]:
df.loc[X_train,'data_type']='train'
df.loc[X_val,'data_type']='validation'



In [22]:
df

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,train
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,validation
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,train


In [23]:
!pip install transformers

     |████████████████████████████████| 1.3MB 12.0MB/s 
     |████████████████████████████████| 1.1MB 56.2MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2dfda3cf072554f99df9f79099b405630f1c585f7e4dac050c71e47f0966e35e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [24]:
!pip3 install --user --upgrade tensorflow  # install in $HOME

     |████████████████████████████████| 320.4MB 54kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [25]:
import tensorflow
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [26]:
tensorflow.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [27]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True,
                                          )

In [29]:
encoded_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True, #Know when sentence is finished
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"
)



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [30]:

encoded_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='validation'].text.values,
    add_special_tokens=True,
    return_attention_mask=True, #Know when sentence is finished
    pad_to_max_length=True,
    max_length=256,
    return_tensors="pt"
)




/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [31]:
input_ids_train=encoded_data_train['input_ids']
attention_mask_train=encoded_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val=encoded_data_val['input_ids']
attention_mask_val=encoded_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='validation'].label.values)


In [32]:
dataset_train=TensorDataset(input_ids_train,attention_mask_train, labels_train)
dataset_val=TensorDataset(input_ids_val,attention_mask_val, labels_val)


In [33]:
len(dataset_train)


1258

In [34]:
len(dataset_val)

223

In [35]:
from transformers import BertForSequenceClassification

In [36]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                      num_labels=len(label_dict),
                                      output_attentions=False,
                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [38]:
batch_size=32

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)


In [39]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [40]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

epochs=10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps= len(dataloader_train)*epochs
)

In [41]:
import numpy as np
from sklearn.metrics import f1_score


In [42]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat,preds_flat, average='weighted')



In [43]:
def accuracy_per_class(preds, labels):
  label_dict_inverse= {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_preds = labels_flat[labels_flat==label]

    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')    

In [44]:
import random

seed_val=17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [45]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

print(device)

cuda


In [51]:
def evaluate(dataloader_val):

  model.eval()

  loss_val_total =0
  predictions, true_vals = [], []

  for batch in dataloader_val:

    batch= tuple(b.to(device) for b in batch)

    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]
              }
    with torch.no_grad():
      outputs=model(**inputs)

    loss=outputs[0]
    logits=outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)

  loss_val_avg= loss_val_total/len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [55]:
for epoch in tqdm(range(1, epochs+1)):
  model.train()

  loss_train_total = 0

  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), 
                      leave=False, disable=False)
  
  for batch in progress_bar:
    model.zero_grad()

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
                }
    outputs= model(**inputs)

    loss=outputs[0]
    loss_train_total += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch)) })

  torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')

  tqdm.write('\nEpoch {epoch}')

  loss_train_avg = loss_train_total/ len(dataloader_train)
  tqdm.write(f'Training Loss: {loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score (weighted): {val_f1}')



FileNotFoundError: ignored

In [53]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                    num_labels=len(label_dict),
                                                    output_attentions=False,
                                                    output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [54]:
model.to(device)
pass

In [ ]:
model.load_state_dict(torch.load())